## Getting resources

We use [xp5k](https://github.com/pmorillon/xp5k) to manage the interaction with the Grid resources.

We create a new ```XP``` and define a new job.

Note that ```resources``` are the same as those given in a ```oarsub``` command line.

In [ ]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
    :resources => ["nodes=3, walltime=1:00:00"],
    :site      => 'lyon',
    :types     => ["allow_classic_ssh"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 3})],
    :command   => "sleep 86400"
    })

## Submission

```submit``` is asynchronous, ```wait_for_jobs``` will wait for all the resources to be ready.

In [ ]:
myxp.submit
myxp.wait_for_jobs

## Run command

```xp5k/rake``` include some helpers to run commands on the nodes of your reservation.

In [ ]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'msimonin'} do 
    cmd = ['uptime -s']
end

## Draw the bar plot

The output of the above command is a map :

 * each key is a server name 
 * a value is the outputs of the commands run on this server
 
e.g : ``` 'sagittaire-1.lyon.grid5000.fr' => ['out1', ... , 'outn'] ```

The following code compute the time since last reboot in minute.

In [ ]:
require 'nyaplot'
require 'time'


now = Time.now.to_i
plot = Nyaplot::Plot.new
df = Nyaplot::DataFrame.new({
    host: h.keys
           .map{|x| x.split('.').first},
    uptime: h.values
             .map{|x| x.chomp}
    .map{|x| ((now - Time.parse(x).to_f)/60).round(2)}
    })

    

In [ ]:
plot=Nyaplot::Plot.new
plot.add_with_df(df, :bar, :host, :uptime)
plot.configure do
    x_label('')
    y_label('Uptime (min)')
    rotate_x_label(-90)
end


In [ ]:
myxp.clean